In [4]:
##%overwritefile
##%file:src/dart_do_execute.py
##%noruncode
    def do_execute(self, code, silent, store_history=True,
                   user_expressions=None, allow_stdin=True):
        try:
            self.silent = silent
            ##扫描代码
            magics, code = self._filter_magics(code)
            if len(self.addkey2dict(magics,'replcmdmode'))>0:
                return self.send_replcmd(code, silent, store_history=True,
                    user_expressions=None, allow_stdin=False)

            if len(self.addkey2dict(magics,'noruncode'))>0
                and ( len(self.addkey2dict(magics,'command'))>0 
                or len(self.addkey2dict(magics,'dartcmd'))>0):
                return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}
            
            if len(self.addkey2dict(magics,'file'))<1:
                magics, code = self._add_main(magics, code)
            return_code=0
            fil_ename=''
            ##调生成文件前用接口
            bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,1,1)
            if bcancel_exec:return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}

            with self.new_temp_file(suffix='.dart',dir=os.path.abspath('')) as source_file:
                source_file.write(code)
                source_file.flush()
                newsrcfilename=source_file.name
                fil_ename=newsrcfilename
                return_code=True
                # Generate new src file
                bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,1,2)
                if bcancel_exec:return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}
                if len(self.addkey2dict(magics,'file'))>1:
                        fil_ename=magics['file'][0]
                    else: fil_ename=source_file.name
                # Generate executable file :being
                # Generate executable file :end
            if len(self.addkey2dict(magics,'noruncode'))>0:
                return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}
            
            ##代码运行前
            bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,3,1)
            if bcancel_exec:return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}
            
            self._write_to_stdout("The process :"+fil_ename+"\n")
            p = self.create_jupyter_subprocess(['dart','--verbose',fil_ename]+ magics['args'],cwd=None,shell=False,env=self.addkey2dict(magics,'env'))
            #p = self.create_jupyter_subprocess([binary_file.name]+ magics['args'],cwd=None,shell=False)
            #p = self.create_jupyter_subprocess([self.master_path, binary_file.name] + magics['args'],cwd='/tmp',shell=True)
            self.g_rtsps[str(p.pid)]=p
            return_code=p.returncode
            ##代码启动后
            bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,3,2)
            if bcancel_exec:return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}
            
            if magics!=None and len(self.addkey2dict(magics,'showpid'))>0:
                self._write_to_stdout("The process PID:"+str(p.pid)+"\n")
            while p.poll() is None:
                p.write_contents(magics)

            # wait for threads to finish, so output is always shown
            p._stdout_thread.join()
            p._stderr_thread.join()
            del self.g_rtsps[str(p.pid)]
            p.write_contents(magics)

            return_code=p.returncode
            bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,3,3)
            if bcancel_exec:return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}
            ##代码运行结束
            self.cleanup_files()
            if p.returncode != 0:
                self._write_to_stderr("[Dart kernel] Executable exited with code {}".format(p.returncode))
        except Exception as e:
            self._log(""+str(e),3)
        return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/dart_do_execute.py created successfully


In [5]:
##%overwritefile
##%file:src/dart_do_dart_command.py
##%noruncode
    def do_dart_command(self,commands=None,cwd=None,magics=None):
        p = self.create_jupyter_subprocess(['dart']+commands,cwd=os.path.abspath(''),shell=False)
        self.g_rtsps[str(p.pid)]=p
        if magics!=None and len(magics['showpid'])>0:
            self._write_to_stdout("The process PID:"+str(p.pid)+"\n")
        while p.poll() is None:
            p.write_contents()
        # wait for threads to finish, so output is always shown
        p._stdout_thread.join()
        p._stderr_thread.join()
        del self.g_rtsps[str(p.pid)]
        p.write_contents()

        if p.returncode != 0:
            self._write_to_stderr("[Dart kernel] Executable exited with code {}".format(p.returncode))
        else:
            self._write_to_stdout("[Dart kernel] Info:dart command success.")
        return


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/dart_do_dart_command.py created successfully


In [ ]:
##%overwritefile
##%file:src/dart_do_flutter_command.py
##%noruncode
    def do_flutter_command(self,commands=None,cwd=None,magics=None):
        p = self.create_jupyter_subprocess(['flutter']+commands,cwd=os.path.abspath(''),shell=False)
        self.g_rtsps[str(p.pid)]=p
        if magics!=None and len(magics['showpid'])>0:
            self._write_to_stdout("The process PID:"+str(p.pid)+"\n")
        while p.poll() is None:
            p.write_contents()
        # wait for threads to finish, so output is always shown
        p._stdout_thread.join()
        p._stderr_thread.join()
        del self.g_rtsps[str(p.pid)]
        p.write_contents()

        if p.returncode != 0:
            self._write_to_stderr("[Dart kernel] Executable exited with code {}".format(p.returncode))
        else:
            self._write_to_stdout("[Dart kernel] Info:flutter command success.")
        return


In [5]:
##%overwritefile
##%file:../jupyter-MyDart-kernel/jupyter_MyDart_kernel/kernel.py
###%file:dart_kernel.py
##%noruncode
#
#   MyDart Jupyter Kernel
#   generated by MyPython
#
##%include:src/head.py

##%include:src/common.py

##%include:src/IREPLWrapper.py
##%include:src/RealTimeSubprocess.py

class DartKernel(Kernel):
    implementation = 'jupyter_MyDart_kernel'
    implementation_version = '1.0'
    language = 'Dart'
    language_version = '2.X.X'
    language_info = {'name': 'text/x-csrc',
                     'mimetype': 'text/x-csrc',
                     'file_extension': '.dart'}
    banner = "Dart kernel.\n" \
             "Uses Dart, compiles in dart, and creates source code files and executables in temporary folder.\n"

    main_head = "\n" \
            "\n" \
            "int main(List<String> arguments){\n"

    main_foot = "\nreturn 0;\n}"
    
##%include:src/comm_attribute.py

    def __init__(self, *args, **kwargs):
        super(DartKernel, self).__init__(*args, **kwargs)
        self.kernelinfo='[MyDart Kernel]'
        self._allow_stdin = True
        self.readOnlyFileSystem = False
        self.bufferedOutput = True
        self.linkMaths = True # always link math library
        self.wAll = True # show all warnings by default
        self.wError = False # but keep comipiling for warnings
        self.files = []
        self.resDir = path.join(path.dirname(path.realpath(__file__)), 'resources')

        self.chk_replexit_thread = Thread(target=self.chk_replexit, args=(self.g_rtsps))
        self.chk_replexit_thread.daemon = True
        self.chk_replexit_thread.start()
        self.init_plugin()
##%include:src/_templateHander.py
##%include:src/_readtemplatefile.py

##%include:src/cleannotes.py
##%include:src/kernel_common.py
##%include:src/_is_specialID.py

##%include:src/repl_listpid.py
##%include:src/chk_replexit.py
##%include:src/cleanup_files.py
##%include:src/new_temp_file.py
##%include:src/log_output.py
##%include:src/_write_display_data.py
##%include:src/_write_to_stdout.py
##%include:src/_write_to_stderr.py
##%include:src/_read_from_stdin.py
##%include:src/readcodefile.py
    #####################################################################
##%include:src/_start_replprg.py
##%include:src/process_output.py
##%include:src/send_replcmd.py
    #####################################################################
##%include:src/do_shell_command.py
    
##%include:src/dart_do_dart_command.py
    
##%include:src/dart_do_flutter_command.py
    
##%include:src/send_cmd.py
##%include:src/create_jupyter_subprocess.py
##%include:src/dart_generate_dartfile.py
##%include:src/compile_with_Python2native.py
##%include:src/_filter_env.py
#####################################################################
##%include:src/dart_filter_magics.py
#####################################################################
##%include:src/_add_main.py
##%include:src/_icallfunc.py
##%include:src/dart_do_execute.py
##%include:src/do_shutdown.py
#####################################################################
##%include:src/pluginmng.py


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/../jupyter-MyDart-kernel/jupyter_MyDart_kernel/kernel.py created successfully
